In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm # 데이터를 불러오는 프로그래스를 확인할때 유용
from datasets import load_dataset # 영화 리뷰 데이터셋 불러오기
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
# 데이터 불러오기
data = load_dataset("nbtpj/movielens-1m-ratings")["train"].shuffle(seed=10).select(range(200000)) # 데이터 20만개만 사용
movielens_df = pd.DataFrame(data)
movielens_df = movielens_df[["user_id", "movie_id", "user_rating"]]

user_ids = movielens_df["user_id"].unique()
user_id_map = {id: index for index, id in enumerate(user_ids)}
movie_ids = movielens_df["movie_id"].unique()
movie_id_map = {id: index for index, id in enumerate(movie_ids)}

movielens_df["user_id"] = movielens_df["user_id"].map(user_id_map)
movielens_df["movie_id"] = movielens_df["movie_id"].map(movie_id_map)

train_data, test_data = train_test_split(movielens_df, test_size=0.2, random_state=10)

추천 알고리즘 모델 학습하기

In [ ]:
num_users = len(user_ids)  # 사용자 수
num_movies = len(movie_ids) # 영화의 수
num_features = 10 # 사용할 특징(Feature)의 수

user_features = np.random.normal(0, .1, (num_users, num_features)) # 사용자 특징을 나타내는 배열을 생성합니다. 배열의 각 요소는 평균이 0, 표준편차가 0.1인 정규분포에서 랜덤하게 선택됩니다.
movie_features = np.random.normal(0, .1, (num_movies, num_features)) # 영화 특징을 나타내는 배열을 생성합니다. 배열의 각 요소는 평균이 0, 표준편차가 0.1인 정규분포에서 랜덤하게 선택됩니다.

learning_rate = 0.01
regularization = 0.1 # 정규화 파라미터 0.1로 설정, L2 Norm2 기법 사용
epochs = 20
epoch_rmse: list[float] = [] # 각 에폭에서의 평균 제곱근 오차(RMSE)를 저장할 리스트를 초기화합니다. Loss 함수로 사용한다.

# 사용자 ID와 영화 ID를 입력받아 해당 사용자와 영화의 특징을 내적하여 평점을 예측하는 함수입니다.
  # 즉, 추천 값을 예측할 때 사용된다.
def predict_rating(user_id: int, movie_id: int) -> float:
  return np.dot(user_features[user_id], movie_features[movie_id])

# 에폭 수만큼 반복하면서 학습을 진행합니다.
for epoch in tqdm(range(epochs)):
  squared_errors: list[float] = []  # 제곱 오류를 저장할 리스트를 초기화합니다.

  # 훈련 데이터의 각 행을 반복하면서 사용자 ID, 영화 ID, 실제 평점을 가져옵니다.
  for _, row in train_data.iterrows():
    user_id = int(row["user_id"])
    movie_id = int(row["movie_id"])
    rating = row["user_rating"] # Ground Truth값

    # 예측 평점을 계산합니다.
    prediction = predict_rating(user_id, movie_id)
    error = rating - prediction  # 실제 평점과 예측 평점의 차이(오류)를 계산합니다.
    squared_errors.append(error**2)  # 오류의 제곱을 squared_errors 리스트에 추가합니다.

    # 사용자 특징과 영화 특징을 업데이트합니다. 업데이트는 경사 하강법을 사용하며, 정규화 항을 포함합니다.
    user_features[user_id] += learning_rate * (error * movie_features[movie_id] - regularization * user_features[user_id])
    movie_features[movie_id] += learning_rate * (error * user_features[user_id] - regularization * movie_features[movie_id])

# 현재 에폭에서의 평균 제곱근 오차(RMSE)를 계산하여 epoch_rmse 리스트에 추가합니다.
  epoch_rmse.append(np.sqrt(np.mean(squared_errors)))


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range(1, epochs+1), epoch_rmse, linewidth=2, color="#fc1c49")
plt.title("Epoch vs. RMSE")
plt.xlabel("Epoch")
plt.ylabel("RMSE")
plt.grid(True)
plt.show()

Validation

In [ ]:
predictions: list[float] = []
true_ratings: list[float] = []

for _, row in tqdm(test_data.iterrows(), total=test_data.shape[0]):
  user_id = int(row["user_id"])
  movie_id = int(row["movie_id"])
  true_rating = row["user_rating"]

  predicted_rating = predict_rating(user_id, movie_id)
  predictions.append(round(predicted_rating))
  true_ratings.append(true_rating)

In [ ]:
plt.figure(figsize=(10,5))
plt.hist(predictions, bins=5, alpha=0.5, label="Predicted", color="#fc1c49")
plt.hist(true_ratings, bins=5, alpha=0.5, label="Actual", color="#00a67d")
plt.title("Predicted vs. Actual Rating Distribution")
plt.xlabel("Rating")
plt.ylabel("Frequency")
plt.legend()
plt.show()

In [ ]:
rmse = np.sqrt(np.mean((np.array(predictions) - np.array(true_ratings))**2))
print(f"RMSE: {rmse}") # RMSE: 1.0052735945999975